# 레진 데이터 챌린지 with 파이콘 한국 2017

http://tech.lezhin.com/events/data-challenge-pyconkr-2017

## 모델 학습 및 평가

학습에 사용할 데이터를 다음처럼 준비하였습니다.

In [1]:
import pandas as pd

df = pd.read_table('lezhin_dataset_v2_training.tsv', header=None)
df_1 = df.drop(df.columns[[6,7,]], axis=1)
df_1 = df_1.dropna(axis=1)

In [2]:
from sklearn.preprocessing import MinMaxScaler

# very important. It does not work without it.
scaler = MinMaxScaler(feature_range=(0, 1))
df_1 = scaler.fit_transform(df_1)
df_1 = pd.DataFrame(df_1)

학습에 사용할 데이터는 전체 사용합니다. 정확도를 판단하기 위해 30% 데이터를 사용합니다.

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
import numpy as np

# 데이터 전처리
rows = len(df_1)
cut_row = rows * 7 // 10
#cut_row = rows
print(cut_row)
x_train = df_1.iloc[:,1:].values.tolist()
y_train = df_1.iloc[:,0].values.tolist()
x_test = df_1.iloc[cut_row:,1:].values.tolist()
y_test = df_1.iloc[cut_row:,0].values.tolist()

# pickle로 저장
import pickle
import os
def save_data(data, fname='data2.pic'):
    if not os.path.isfile(fname):
        f = open(fname, 'wb')
        pickle.dump(data, f)
        f.close()
    
def load_data(fname='data2.pic'):
    f = open(fname, 'rb')
    x_train,y_train,x_test,y_test = pickle.load(f)
    f.close()
    return x_train,y_train,x_test,y_test
    
save_data([x_train,y_train,x_test,y_test])
x_train, y_train, x_test, y_test = load_data()

Using TensorFlow backend.


455675


binary crossentropy를 사용하여 학습합니다.

In [4]:
model = Sequential()
model.add(Dense(1, input_dim=147, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.1), metrics=['accuracy'])

model.summary()
model.fit(x_train, y_train, epochs=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 148       
Total params: 148
Trainable params: 148
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
650965/650965 [==============================] - 54s - loss: 0.5262 - acc: 0.7354    
Epoch 2/10
650965/650965 [==============================] - 51s - loss: 0.5214 - acc: 0.7396    
Epoch 3/10
650965/650965 [==============================] - 60s - loss: 0.5209 - acc: 0.7404    
Epoch 4/10
650965/650965 [==============================] - 62s - loss: 0.5207 - acc: 0.7402    
Epoch 5/10
650965/650965 [==============================] - 51s - loss: 0.5205 - acc: 0.7405    
Epoch 6/10
650965/650965 [==============================] - 52s - loss: 0.5204 - acc: 0.7408    
Epoch 7/10
650965/650965 [==============================] - 50s - loss: 0.5204 - acc: 0.74

해당 모델을 사용하여 정확도를 구합니다.
정확도는 약 67.64%가 나왔습니다.

In [5]:
loss_and_metrics = model.evaluate(x_test, y_test)

print('')
print('loss_and_metrics : ' + str(loss_and_metrics))

194496/195290 [============================>.] - ETA: 0s
loss_and_metrics : [0.56578984513517916, 0.67638896000838367]


4개의 relu 계층과 1개의 sigmoid 계층을 사용하여 학습합니다.

In [6]:
from keras.optimizers import Adam

model = Sequential()
model.add(Dense(100, input_dim=147, init='uniform', activation='relu'))
model.add(Dense(100, init='uniform', activation='relu'))
model.add(Dense(100, init='uniform', activation='relu'))
model.add(Dense(50, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))

#sgd = SGD()
#model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.1), metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()
model.fit(x_train, y_train, epochs=100)

/Users/donglyeolsin/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, kernel_initializer="uniform", input_dim=147, activation="relu")`
/Users/donglyeolsin/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, kernel_initializer="uniform", activation="relu")`
/Users/donglyeolsin/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, kernel_initializer="uniform", activation="relu")`
/Users/donglyeolsin/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, kernel_initializer="uniform", activation="relu")`
/Users/donglyeolsin/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="uniform"

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               14800     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 51        
Total params: 40,101
Trainable params: 40,101
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
650965/650965 [==============================] - 119s - loss: 0.4386 - acc: 0.7938   
Epoch 2/100
650965/650965 [=============

해당 모델을 사용하여 정확도를 구합니다.
정확도는 약 87.29%가 나왔습니다.

In [7]:
loss_and_metrics = model.evaluate(x_test, y_test)

print('')
print('loss_and_metrics : ' + str(loss_and_metrics))

194560/195290 [============================>.] - ETA: 0s
loss_and_metrics : [0.3222619390542914, 0.87285575298030194]


해당 모델을 저장합니다.

In [8]:
# 모델 저장하기
from keras.models import load_model
model.save('lezhin.h5')